
## First time-based data using Prophet

- still I don't use parameter tuning tool ex) optuna gridCV

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cudf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'cudf'

## Simle store split & basic prophet ##

당장 생각나는 방법은 store별로 나눠서 df 생성 후 각각 prophet으로 만들고 넣기..

> 합체는 id기준으로 해야할텐데, 어떻게 하는게 가장 효율적일지.. 우선 data 에서 store_id의 순서를 알 필요가 있을듯

In [ ]:
train = cudf.read_csv("/kaggle/input/devkor-sales-prediction/train.csv")
test = cudf.read_csv("/kaggle/input/devkor-sales-prediction/test.csv")
oil = pd.read_csv("/kaggle/input/devkor-sales-prediction/oil.csv")
holidays = pd.read_csv("/kaggle/input/devkor-sales-prediction/holidays.csv")
stores = pd.read_csv("/kaggle/input/devkor-sales-prediction/store.csv")
transactions = pd.read_csv("/kaggle/input/devkor-sales-prediction/transactions.csv")
submission = pd.read_csv("/kaggle/input/devkor-sales-prediction/sample_submission.csv")

In [ ]:
train.tail()

In [ ]:
test.tail()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
%matplotlib inline
warnings.filterwarnings('ignore')

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly


In [ ]:
p_list = train['product_type'].unique().to_pandas().tolist()
s_list = train['store_id'].unique().to_pandas().tolist()

In [ ]:
stores = []
for i in s_list:
    stores.append(train.loc[train['store_id'] == i])

In [ ]:
stores[0].head()

In [ ]:
s_list.sort()

In [ ]:
len(s_list), len(p_list)

In [ ]:
type(train)

In [ ]:
from tqdm import tqdm

In [ ]:
storess = []
for i in tqdm(s_list):
    storess.append({})
    for j in p_list:
        storess[i-1][j] = train.loc[(train['store_id'] == i) & (train['product_type'] == j)]
        
    

In [ ]:
storess[0]['AUTOMOTIVE'].tail()

In [ ]:
test.loc[:, 'sales'] = 0.0
test.loc[:, 'date'] = pd.to_datetime(test['date'].to_pandas())
test.head()

*  의도한대로 store_id와 product_type 별로 잘 데이터가 분리가 되었다.
*  각 fitting을 시켜보자.

In [ ]:
test.loc[(test['store_id']==1) & (test['product_type']=='AUTOMOTIVE')]

In [ ]:
# m = Prophet(changepoint_prior_scale=0.3)

# df = storess[0]['AUTOMOTIVE'][['date', 'sales']]
# df.columns = ['ds', 'y']
# m.fit(df.to_pandas())
        
# future = m.make_future_dataframe(periods=15)
# y_pred = m.predict(future)
# predict = y_pred.tail(15)



In [ ]:
# test.loc[(test['store_id']==1) & (test['product_type']=='AUTOMOTIVE'), "sales"]

In [ ]:
# test.loc[(test['store_id']==1) & (test['product_type']=='AUTOMOTIVE'), "sales"] = predict['yhat'].to_list()

In [ ]:
# test.loc[(test['store_id']==1) & (test['product_type']=='AUTOMOTIVE'), "sales"]

In [ ]:
class supprpredictdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

In [ ]:
for i in tqdm(range(len(s_list))):
    for j in p_list:
        df = storess[i][j][['date', 'sales']]
        df.columns = ['ds', 'y']
        with suppress_stdout_stderr():
            m = Prophet(changepoint_prior_scale=0.3)
            m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
            m.fit(df.to_pandas())

        future = m.make_future_dataframe(periods=15)
        y_pred = m.predict(future)
        predict = y_pred.tail(15)
        test.loc[(test['store_id']==i+1) & (test['product_type']==j), "sales"] = predict['yhat'].to_list()

In [ ]:
test.loc[(test['store_id']==1) & (test['product_type']==p_list[7]), "sales"]

In [ ]:
submission.loc[:, "sales"] = test.to_pandas().loc[:, "sales"].to_list()
submission.to_csv("submission.csv", index=False)